In [ ]:
### CODE FOR EXTRACTING COORDINATES OF LOCATION ####

import pandas as pd
from geopy.geocoders import Nominatim

# Reading Dataset on Houses
Houses = pd.read_csv("Houses_2023.csv")

# Extracting Required Features from the Dataset
Area = Houses.iloc[:, 0]
City = Houses.iloc[:, 1]

for i in range(0, len(Area)-1):

  # Setting up the Agent
  Locator_agent = Nominatim(user_agent="location_details")

  # Finding Coordinates
  location = Locator_agent.geocode(Area[i]+","+ City[i], timeout=None)

  # If not successful, Pass
  if location is None:
    pass

  # Else, Update the file
  else:
    print(i+1)
    Houses["Latitude"][i] = location.latitude
    Houses["Longitude"][i] = location.longitude

# Saving File
Houses.to_csv("houses_2023.csv", index=False)

In [18]:
### CODE FOR TRAINING THE MODELS ####

# Importing Required Libraries
from geopy.geocoders import Nominatim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import folium


# Function to get coordinates of a location
def get_coordinates(loc):

  # Setting up the Agent
  Locator_agent = Nominatim(user_agent="location_details")

  # Finding Coordinates
  location = Locator_agent.geocode(loc, timeout=None)

  # If not found
  if location is None:
    print("No Such Location Found. Please Try Another Location")

  else:
      # Returning Coordinates:
      return location.latitude, location.longitude

# Function to find Optimum location Based on Price
def Locations_on_Price(Houses, price):

  index = []

  # filtering data based on Price
  for i in range(0, len(Houses) - 1):
    if Houses["Rent"][i] < price:
      index.append(i)
  
  return index

# Function to Find the Best Location for User
def find_nearest_neighbours(Houses, lat, lon):
  
  # Extracting Required Features from the Dataset
  features = Houses.loc[:,["Latitude", "Longitude"]]
  y = Houses.loc[:,["neighbourhood"]]

  # Implementing K nearest neighbous classifier
  # Here, K = 500
  K = 150
  knn = KNeighborsClassifier(K)

  # Fitting the Model
  knn.fit(features.values, y.values.ravel())

  # Finding Nearest Locations from the User's Preferred Location:
  i = knn.kneighbors([[lat, lon]], n_neighbors = K, return_distance = False)

  # Converting index to a 1-d Array
  index = i.ravel()
  return index

# Function for Showing the Results
def show_picks(index):

  # Showing 5 Best Locations to the User
  print("\nHere Are Our 5 Top Picks For You!\n")
  print("Sr")

  for i in range(0, len(index)):

    if i == 5:
      break

    print(i+1, "\t", Houses["neighbourhood"][index[i]], ",", Houses["neighbourhood_group"][index[i]], ","
          , Houses["Area"][index[i]], "Marla", "==>", Houses["Rent"][index[i]], "k")
  print("\n")

def Display_Maps(lat, lon, index):

  # Creating Map Instance
  map = folium.Map(location=[lat[0],lon[0]], zoom_start = 12)

  # Displaying all locations on map
  for num in range(0, len(index)):
    
    if num == 5:
      break

    map.add_child(folium.Marker(location=[Houses["Latitude"][index[num]] + num/1000, 
                                        Houses["Longitude"][index[num]] + num/1000], popup=Houses["neighbourhood"][index[num]], 
                                        icon=folium.Icon(color='blue')))

  # Adding preferred locations
  for i in range(0, 3):
    map.add_child(folium.Marker(location=[lat[i],lon[i]], popup="P"+str(i+1),
                                icon=folium.Icon(color='red')))
  
  # Displaying Map
  display(map)

# Function for find common area from all preferred locations
def find_common_area(indices):

  c = [value for value in indices[0] if value in indices[1]]
  common = [value for value in c if value in indices[2]]

  return common

# Function for Finding Intersection of Common Areas and Area within Price Range
def find_common_price_area(common_areas, price_areas):

  common = [value for value in common_areas if value in price_areas]
  return common


if __name__ == "__main__":

  # Defining Required Columns
  req_cols = ["neighbourhood", "neighbourhood_group", "Area", "Latitude", "Longitude", "Rent"]

  # Reading Dataset on Houses
  Houses = pd.read_csv("Houses_2023.csv", usecols = req_cols)

  # To Store preferred locations
  loc = []
  indices = []
  lat = []
  lon = []

  for i in range(0, 3):

    # Getting Location From the User:
    area = input("Enter Your " + str(i+1) + " Preferred Location: ")
    loc.append(area)
    
    # Finding Coordinates of Location:
    latitude, longitude = get_coordinates(loc[i])
    lat.append(latitude)
    lon.append(longitude)

    # Using K-Nearest Neighbour Model to find Nearest Locations:
    index = find_nearest_neighbours(Houses, lat[i], lon[i])
    indices.append(index)
  
  # Getting Price From User
  price = int(input("Enter Max Price (k): "))
  
  # Finding Best Locations Based on Price
  price_index = Locations_on_Price(Houses, price)

  # Finding Common locations from each preferred locations
  common = find_common_area(indices)

  # Finding Intersection of Common Areas and Area within Price Range
  final_common_areas = find_common_price_area(common, price_index)

  # If no common rental property exist between all three preferred locations
  if len(final_common_areas) == 0:
    print("\n*** No Result Matches Your Query. Please Try Something Else :) ***")
  
  else:
      # Show the Best Stay Locations to User
      show_picks(final_common_areas)

  # Plotting Maps
  Display_Maps(lat, lon, final_common_areas)

Enter Your 1 Preferred Location: H-12 Islamabad
Enter Your 2 Preferred Location: I-10 Islamabad
Enter Your 3 Preferred Location: I-11 Islamabad
Enter Max Price (k): 90
[1144, 1145, 1139, 1154, 1720, 61, 64, 63, 239, 238, 219, 220, 148, 146, 59, 58, 60, 213, 389]

Here Are Our 5 Top Picks For You!

Sr
1 	 Bahria Enclave - Sector N ,  Islamabad , 5.0 Marla ==> 70.0 k
2 	 Bahria Enclave - Sector N ,  Islamabad , 5.0 Marla ==> 70.0 k
3 	 Bahria Main Boulevard ,  Rawalpindi , 7.0 Marla ==> 75.0 k
4 	 Bahria Town Phase 8 - Rafi Block ,  Rawalpindi , 5.0 Marla ==> 60.0 k
5 	 G-11 ,  Islamabad , 3.3 Marla ==> 50.0 k


